In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('working.csv')
df.head(3)

,txn_type,date,eth,punk_id,type,accessories,type_rarity,type_pct,num_accessories
0,Sold,2017-06-23,0.98,0,Female,"['Green Eye Shadow', 'Earring', 'Blonde Bob']",2.6,0.384,3
1,Bid,2017-06-23,0.98,0,Female,"['Green Eye Shadow', 'Earring', 'Blonde Bob']",2.6,0.384,3
2,Bid,2017-06-23,0.67,0,Female,"['Green Eye Shadow', 'Earring', 'Blonde Bob']",2.6,0.384,3


In [3]:
df.txn_type.value_counts()

Offered            64170
Bid                23880
Sold               18626
Bid Withdrawn      15282
Transfer            1234
Offer Withdrawn      413
Claimed              121
(Unwrap)              72
(Wrap)                50
Name: txn_type, dtype: int64

In [4]:
#subsetting dataframe by txn type

In [5]:
bid = df.loc[df.txn_type == 'Bid']
bid.shape

(23880, 9)

In [6]:
sale = df.loc[df.txn_type == 'Sold']
sale.shape

(18626, 9)

In [7]:
sale_mlb = sale.drop(columns=['txn_type', 'date', 'punk_id', 'type', 'type_pct'])
sale_mlb.head(3)

,eth,accessories,type_rarity,num_accessories
0,0.98,"['Green Eye Shadow', 'Earring', 'Blonde Bob']",2.6,3
3,0.25,"['Black Lipstick', 'Green Eye Shadow', 'Mohawk...",2.6,3
6,0.20,"['Purple Eye Shadow', 'Wild Blonde', 'Cigarette']",2.6,3


In [9]:
sale_mlb.accessories = sale_mlb.accessories.map(eval)

In [10]:
#attempting to multilabelbinarize accessories column
from sklearn.preprocessing import MultiLabelBinarizer

In [12]:
mlb = MultiLabelBinarizer()
mlb.fit(sale_mlb['accessories'])

acc_mlb = pd.DataFrame(mlb.transform(sale_mlb['accessories']), columns=mlb.classes_)

In [17]:
data = pd.concat([sale_mlb, acc_mlb], axis=1)
data.head()

,eth,accessories,type_rarity,num_accessories,3D Glasses,Bandana,Beanie,Big Beard,Big Shades,Black Lipstick,...,Tassle Hat,Tiara,Top Hat,VR,Vampire Hair,Vape,Welding Goggles,Wild Blonde,Wild Hair,Wild White Hair
0,0.98,"[Green Eye Shadow, Earring, Blonde Bob]",2.6,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.25,"[Black Lipstick, Green Eye Shadow, Mohawk Thin]",2.6,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [21]:
data.dropna(inplace=True)

In [22]:
#Define X and y
X= data.drop(columns=['accessories', 'eth'])
y= data['eth']

#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [23]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train), lr.score(X_test, y_test)

(0.29808384880767547, 0.19812665486575465)

In [25]:
lr.coef_.max()

0.47198407639354806

In [26]:
lr.coef_.min()

-0.17291754909447665

In [27]:
lr.coef_.std()

0.1128766434842977

In [28]:
from sklearn.preprocessing import StandardScaler

z=StandardScaler()
Z_train = z.fit_transform(X_train)
Z_test = z.transform(X_test)

In [35]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(Z_train, y_train)
lr.score(Z_train, y_train), lr.score(Z_test, y_test)

(0.29808384880767536, 0.19812665486575487)

In [29]:
from sklearn.neighbors import KNeighborsRegressor
kn = KNeighborsRegressor()
kn.fit(Z_train, y_train)
kn.score(Z_train, y_train), kn.score(Z_test, y_test)

(0.4003313908663466, -0.03069689015403254)

In [30]:
from sklearn.tree import DecisionTreeRegressor
dr = DecisionTreeRegressor(random_state=42)
dr.fit(X_train, y_train)
dr.score(X_train, y_train), dr.score(X_test, y_test)

(0.817338757319835, 0.15460052722875983)

In [31]:
from sklearn.ensemble import BaggingRegressor
br = BaggingRegressor(random_state=42)
br.fit(X_train, y_train)
br.score(X_train, y_train), br.score(X_test, y_test)

(0.6643612924318967, 0.006383128519303338)

In [32]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
rf.score(X_train, y_train), rf.score(X_test, y_test)

(0.7369066457892757, 0.22307726435850417)

In [33]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(random_state=42)
ada.fit(X_train, y_train)
ada.score(X_train, y_train), ada.score(X_test, y_test)

(0.6638070042019315, 0.18039934621954756)

In [34]:
from sklearn.svm import LinearSVR
svr = LinearSVR()
svr.fit(Z_train, y_train)
svr.score(Z_train, y_train), svr.score(Z_test, y_test)

/Users/kevingates/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


(0.26167517764278103, 0.20850093372073686)